## 人名分类问题：
- 以一个人名为输入，使用模型帮助我们判断它最有可能是来自哪一个国家的人名，这在某些国际化公司的业务中具有重要意义，在用户注册过程中，会根据用户填写的名字直接给他分配可能的国家或地区选项，以及该国家或地区的国旗，限制手机号吗位数等等

## 整个案例分为5个步骤
- 第一步：导入必要的工具包
- 第二步: 对data文件中的数据进行处理，满足训练要求
- 第三步: 构建RNN模型(包括传统的RNN,LSTM以及GRU等)
- 第四步：构建训练函数并进行训练
- 第五步：构建评估函数并进行预测

In [13]:
#  第一步：导入必要的工具包
from io import open
import glob
import os

import string
import unicodedata

import random
import time
import match
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


# 第二步：对data文件中的数据进行处理，满足训练要求
# 获取所有的常用字符包括字母和常用标点
all_letters = string.ascii_letters + " .,;'"
# 获取常用字符数量
n_letters = len(all_letters)
print("n_letter:", n_letters)
print("all_letters:", all_letters)

n_letter: 57
all_letters: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'


#### 字符规范之unicode转Ascii函数

In [2]:
# 关于编码问题我们暂且不去考虑
# 这个函数的作用就是去掉一些语言中的重音标记
# 如: Ślusàrski ---> Slusarski
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
# 调用
s = "Ślusàrski"
a = unicodeToAscii(s)
print(a)


Slusarski


In [4]:
# 构建一个从持久化文件中读取内容到内存的函数
data_path = "./data/names/"

def readLines(filename):
    """从文件中读取每一行加载到内存中形成列表"""
    # 打开指定文件并读取所有内容, 使用strip()去除两侧空白符, 然后以'\n'进行切分
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    # 对应每一个lines列表中的名字进行Ascii转换, 使其规范化.最后返回一个名字列表
    return [unicodeToAscii(line) for line in lines]

# filename是数据集中某个具体的文件, 我们这里选择Chinese.txt
filename = data_path + "Chinese.txt"
lines = readLines(filename)
print(lines)

['Ang', 'AuYong', 'Bai', 'Ban', 'Bao', 'Bei', 'Bian', 'Bui', 'Cai', 'Cao', 'Cen', 'Chai', 'Chaim', 'Chan', 'Chang', 'Chao', 'Che', 'Chen', 'Cheng', 'Cheung', 'Chew', 'Chieu', 'Chin', 'Chong', 'Chou', 'Chu', 'Cui', 'Dai', 'Deng', 'Ding', 'Dong', 'Dou', 'Duan', 'Eng', 'Fan', 'Fei', 'Feng', 'Foong', 'Fung', 'Gan', 'Gauk', 'Geng', 'Gim', 'Gok', 'Gong', 'Guan', 'Guang', 'Guo', 'Gwock', 'Han', 'Hang', 'Hao', 'Hew', 'Hiu', 'Hong', 'Hor', 'Hsiao', 'Hua', 'Huan', 'Huang', 'Hui', 'Huie', 'Huo', 'Jia', 'Jiang', 'Jin', 'Jing', 'Joe', 'Kang', 'Kau', 'Khoo', 'Khu', 'Kong', 'Koo', 'Kwan', 'Kwei', 'Kwong', 'Lai', 'Lam', 'Lang', 'Lau', 'Law', 'Lew', 'Lian', 'Liao', 'Lim', 'Lin', 'Ling', 'Liu', 'Loh', 'Long', 'Loong', 'Luo', 'Mah', 'Mai', 'Mak', 'Mao', 'Mar', 'Mei', 'Meng', 'Miao', 'Min', 'Ming', 'Moy', 'Mui', 'Nie', 'Niu', 'OuYang', 'OwYang', 'Pan', 'Pang', 'Pei', 'Peng', 'Ping', 'Qian', 'Qin', 'Qiu', 'Quan', 'Que', 'Ran', 'Rao', 'Rong', 'Ruan', 'Sam', 'Seah', 'See ', 'Seow', 'Seto', 'Sha', 'Shan', 'Sh

#### 构建人名类别(所属的语言)列表和人名对应关系字典

In [5]:
# 构建的category_lines形如：{"English":["Lily", "Susan", "Kobe"], "Chinese":["Zhang San", "Xiao Ming"]}
category_lines = {}
# all_categories形如： ["English",...,"Chinese"]
all_categories = []

# 读取指定路径下的txt文件，使用glob,path中可以使用正则表达式
for filename in glob.glob(data_path + "*.txt"):
    # 获取每个文件的文件名，就是对应的名字类别
    category = os.path.splitext(os.path.basename(filename))[0]
    # 将其逐一加到all_categories列表中
    all_categories.append(category)
    # 然后读取每个文件的内容，形成名字列表
    lines = readLines(filename)
    # 按照对应的类别，将名字列表写入到catagory_lines字典中
    category_lines[category] = lines

# 查看类别总数
n_categories = len(all_categories)
print("n_categories:", n_categories)

n_categories: 18


In [6]:
# 随便查看其中的一些内容
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [7]:
# 将人名转化为对应的one-hot张量表示
# 将字符串(单词粒度)转化为张量表示，如："ab" --->
# tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0.]],

#        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0.]]])
def lineToTensor(line):
    """将人名转化为对应的one-hot张量表示，参数line是输入的人名"""
    # 首先初始化一个0张量，它的形状(len(line), 1, n_letters)
    # 代表人名中的每个字母用一个1 x n_letters的张量表示
    tensor = torch.zeros(len(line), 1, n_letters)
    # 遍历这个人名中的每个字符索引和字符
    for li, letter in enumerate(line):
        # 使用字符串方法find找到每个字符在all_letters中的索引
        # 它也是我们生成one-hot张量中1的索引位置
        tensor[li][0][all_letters.find(letter)] = 1
    # 返回结果
    return  tensor


line = "Bai"
line_tensor = lineToTensor(line)
print("line_tensor:", line_tensor)

line_tensor: tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]]])


In [8]:
# 构建传统RNN模型
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        """初始化函数中有4个参数，分别代表RNN输入最后一维尺寸，RNN隐层最后一维尺寸,RNN层数"""
        # input_size:代表RNN输入的最后一个维度
        # hidden_size:代表RNN隐藏层的最后一个维度
        # output_size:代表RNN网络最后线性层的输出维度
        # num_layers:代表RNN网络的层数
        super(RNN, self).__init__()
        # 将hidden_size与num_layers传入其中
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # 实例化预定义的nn.RNN,它的三个参数分别是input_size，hidden_size, num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers)
        # 实例化nn.Linear,这个线性层用于将nn.RNN的输出值维度转化为指定的输出维度
        self.linear = nn.Linear(hidden_size, output_size)
        # 实例化nn中预定的Softmax层，用于从输出层获得类别结果
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input, hidden):
        """完成传统RNN的主要逻辑，输入参数input代表输入张量，它的形状是1 x n_letters
        hidden代表RNN的隐层张量，它的形状是self.num_layers x 1 x self.hidden_size"""
        # 因此预定义的nn.RNN要求输入维度一定是三维张量，因此在这里使用unsqueeze(0)扩展一个维度
        input = input.unsqueeze(0)
        # 将input和hidden输入到传统RNN的实例化对象中，如果num_layers = 1，rr恒等于hn
        rr, hn = self.rnn(input, hidden)
        # 将从RNN中获得的结果通过线性变换和softmax返回，同时返回hn作为后续RNN的输入
        return self.softmax(self.linear(rr)), hn

    def initHidden(self):
        """初始化隐层张量"""
        # 初始化一个(self.num_layers, 1, self.hidden_size)形状为0的张量
        return torch.zeros(self.num_layers, 1, self.hidden_size)

In [16]:
# torch.unsqueeze演示
x = torch.tensor([1, 2, 3, 4])
print(x.shape)
z = torch.unsqueeze(x, 0)
print(z.shape)

torch.Size([4])
torch.Size([1, 4])


In [17]:
y = torch.unsqueeze(x, 1)
print(y.shape)

torch.Size([4, 1])


In [11]:
# 构建LSTM模型
# 使用nn.LSTM构建完成LSTM使用类
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        # 初始化函数的参数与传统RNN相同
        super(LSTM, self).__init__()
        # 将hidden_size与num_layers传入其中
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # 实例化预定义的nn.LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        # 实例化nn.Linear,这个线性层用于将nn.LSTM的输出维度转化为指定的输出维度
        self.linear = nn.Linear(hidden_size, output_size)
        # 实例化nn中预定的Softmax层，用于从输出层获得类别结果
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input, hidden, c):
        """在主要逻辑函数中多出一个参数c, 也就是LSTM中的细胞状态张量"""
        # 使用unsqueeze(0)扩展一个维度
        input = input.unsqueeze(0)
        # 将input, hidden以及初始化的c传入lstm中
        rr, (hn, c) = self.lstm(input, (hidden, c))
        # 最后返回处理后rr, hn, c
        return self.softmax(self.linear(rr)), hn, c

    def initHiddenAndC(self):
        """初始化函数不仅初始化hidden还要初始化细胞状态c，它们的形状相同"""
        c = hidden = torch.zeros(self.num_layers, 1, self.hidden_size)
        return  hidden, c

In [12]:
# 构建GRU模型
# 使用nn.GRU构建完成传统RNN使用类

# GRU与传统RNN的外部形式相同, 都是只传递隐层张量, 因此只需要更改预定义层的名字
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # 实例化预定义的nn.GRU, 它的三个参数分别是input_size, hidden_size, num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input, hidden):
        input = input.unsqueeze(0)
        rr, hn = self.gru(input, hidden)
        return self.softmax(self.linear(rr)), hn

    def initHidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size)

#### 实例化参数

In [26]:
# 因为是onehot编码, 输入张量最后一维的尺寸就是n_letters
input_size = n_letters

# 定义隐层的最后一维尺寸大小
n_hidden = 57

# 输出尺寸为语言类别总数n_categories
output_size = n_categories

# num_layer使用默认值, num_layers = 1

#### 输入参数

In [27]:
# 假如我们以一个字母B作为RNN的首次输入, 它通过lineToTensor转为张量
# 因为我们的lineToTensor输出是三维张量, 而RNN类需要的二维张量
# 因此需要使用squeeze(0)降低一个维度
input = lineToTensor('B').squeeze(0)

# 初始化一个三维的隐层0张量, 也是初始的细胞状态张量
hidden = c = torch.zeros(1, 1, n_hidden)

#### 调用

In [29]:
rnn = RNN(n_letters, n_hidden, n_categories)
lstm = LSTM(n_letters, n_hidden, n_categories)
gru = GRU(n_letters, n_hidden, n_categories)

rnn_output, next_hidden = rnn(input, hidden)
print("rnn:", rnn_output)
print("rnn.shape:", rnn_output.shape)
lstm_output, next_hidden, c = lstm(input, hidden, c)
print("lstm:", lstm_output)
print("lstm.shape:", lstm_output.shape)
gru_output, next_hidden = gru(input, hidden)
print("gru:", gru_output)
print("gru.shape:", gru_output.shape)

rnn: tensor([[[-2.9879, -3.1140, -2.9302, -2.8892, -3.0818, -2.7007, -2.6251,
          -2.8833, -2.8417, -2.9852, -2.9925, -3.0889, -2.9286, -2.7733,
          -2.7649, -2.8631, -2.9344, -2.7964]]], grad_fn=<LogSoftmaxBackward0>)
rnn.shape: torch.Size([1, 1, 18])
lstm: tensor([[[-2.9077, -2.7731, -2.9073, -2.9322, -2.9242, -2.9533, -2.8012,
          -2.7922, -2.7910, -2.8518, -2.9046, -3.0164, -2.9827, -2.9458,
          -2.9559, -2.7684, -2.8808, -2.9921]]], grad_fn=<LogSoftmaxBackward0>)
lstm.shape: torch.Size([1, 1, 18])
gru: tensor([[[-2.7727, -2.7887, -2.9656, -2.9033, -2.9723, -2.9675, -2.8865,
          -2.7472, -2.9438, -2.8640, -3.0225, -2.8290, -3.0852, -2.9195,
          -2.8837, -2.7212, -2.9279, -2.9047]]], grad_fn=<LogSoftmaxBackward0>)
gru.shape: torch.Size([1, 1, 18])


#### 第四步：构建训练函数并进行训练

In [30]:
# 从输出结果中获得指定类别的函数

def categoryFromOutput(output):
    """从输出结果中获得指定类别，参数为输出张量output"""
    # 从输出张量中返回最大的值和索引，作为我们的类别信息，我们这里主要需要这个索引
    top_n, top_i = output.topk(1)
    # top_i对象中取出索引的值
    category_i = top_i[0].item()
    # 根据索引获得对应语言类型，返回语言类别和索引值
    return all_categories[category_i], category_i

In [33]:
# torch.topk演示
x = torch.arange(1., 6.)
print(x)
z = torch.topk(x, 3)
print(z)

tensor([1., 2., 3., 4., 5.])
torch.return_types.topk(
values=tensor([5., 4., 3.]),
indices=tensor([4, 3, 2]))


In [34]:
# 输入参数
# 将上一步中grur的输出作为函数的输入
output = gru_output
# 调用
category, category_i = categoryFromOutput(output)
print("category:", category)
print("category_i:", category_i)

category: Dutch
category_i: 15


### 随机生成训练数据

In [35]:
def randomTrainingExample():
    """该函数用于随机产生训练数据"""
    # 首先使用random的choice方法从all_categories随机选择一个类别
    category = random.choice(all_categories)
    # 然后再通过category_lines字典取category类别对应的名字列表
    # 之后再从列表中随机取一个名字
    line = random.choice(category_lines[category])
    # 接着将这个类别在所有类别列表中的索引封装成tensor,得到类别张量category_tensor
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    # 最后，将随机取到的名字通过函数lineToTensor转化为onehot张量表示
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

In [36]:
# 我们随机取出十个进行结果查看
for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line, '/ category_tensor =', category_tensor)

category = Arabic / line = Haddad / category_tensor = tensor([2])
category = Spanish / line = Rosa / category_tensor = tensor([10])
category = Portuguese / line = Crespo / category_tensor = tensor([13])
category = Vietnamese / line = Quyen / category_tensor = tensor([5])
category = Italian / line = Palmisano / category_tensor = tensor([12])
category = Dutch / line = Rompaeij / category_tensor = tensor([15])
category = Greek / line = Chrysanthopoulos / category_tensor = tensor([11])
category = Polish / line = Rudawski / category_tensor = tensor([17])
category = Russian / line = Abelsky / category_tensor = tensor([6])
category = Spanish / line = Morales / category_tensor = tensor([10])


#### 构建传统RNN训练函数

In [37]:
# 定义损失函数为nn.NLLLoss，因为RNN的最后一层是nn.LogSoftmax,两者的内部计算逻辑正好能够吻合
criterion = nn.NLLLoss()

# 设置学习率
learning_rate = 0.005

def trainRNN(category_tensor, line_tensor):
    """定义训练函数，它的两个参数是category_tensor类别的张量表示，相当于训练数据的标签
    line_tensor名字张量的表示，相当于对应训练数据"""
    # 在函数中，首先通过实例化对象rnn初始化隐层张量
    hidden = rnn.initHidden()
    # 然后将模型结构中的梯度归0
    rnn.zero_grad()

    # 下面开始进行训练，将训练数据line_tensor的每个字符逐个传入rnn之中，得到最终结果
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    # 因为我们的rnn对象由nn.RNN实例化得到，最终输出形状是三维张量，为了满足于category_tensor
    # 进行对比计算损失, 需要减少第一个维度, 这里使用squeeze()方法
    loss = criterion(output.squeeze(0), category_tensor)

    # 损失进行反向传播
    loss.backward()
    # 更新模型中所有的参数
    for p in rnn.parameters():
        # 将参数的张量表示与参数的梯度乘以学习率的结果相加以此来更新参数
        p.data.add_(-learning_rate, p.grad.data)
    # 返回结果和损失的值
    return output, loss.item()